# Setup

On google colab, you have to restart runtime after running the following line

In [40]:
!pip install omegaconf

In [41]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [42]:
##This is added because my Rdata uses Cdata for the covid data set.
##Rdata use Cdata function to load the data set, then convert it to regression problem
import os
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset')


In [43]:
# folder_name = os.getcwd()

In [44]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [45]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# NCA and LMNN setup

In [47]:
pip install metric-learn

In [48]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'


Newly added data sets for mental health (psychology):

Classification:
'psych_depression_physical_symptons',
'covid_anxious',
'covid_depressed'


In [49]:
dataset_name = 'covid_anxious'
cfg = conf_file['dataset'][dataset_name]
#TODO need to add other covid data sets here.
if dataset_name in ['covid_depressed','covid_anxious','psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)


In [50]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

# Classification with NNKNN

In [51]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")


Unique values: [0 1 2 3]
Counts: [1651 1651 1651 1651]
Xs.size(): torch.Size([6604, 161])


In [52]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  model = Cmodel.NN_k_NN(X_train,
                         y_train,
                         cfg.ca_weight_sharing,
                         cfg.top_case_enabled,
                         cfg.top_k,
                         cfg.discount,
                         device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [53]:
def load_model_cls(X_train,y_train,cfg):
  # Define the model architecture
  model = Cmodel.NN_k_NN(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model

In [54]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = False

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)
  break

print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


Epoch [2/1000], Loss: 1.8530
Epoch [4/1000], Loss: 1.7258
Epoch [6/1000], Loss: 1.6130
Epoch [8/1000], Loss: 1.7196
Epoch [10/1000], Loss: 1.5507
Epoch [12/1000], Loss: 1.3509
Epoch [14/1000], Loss: 1.3829
Epoch [16/1000], Loss: 1.3458
Epoch [18/1000], Loss: 1.3779
Epoch [20/1000], Loss: 1.3624
Epoch [22/1000], Loss: 1.3009
Epoch [24/1000], Loss: 1.2952
Epoch [26/1000], Loss: 1.2926
Epoch [28/1000], Loss: 1.3299
Epoch [30/1000], Loss: 1.3454
Epoch [32/1000], Loss: 1.3334
Epoch [34/1000], Loss: 1.3323
Epoch [36/1000], Loss: 1.3043
Epoch [38/1000], Loss: 1.3009
Epoch [40/1000], Loss: 1.3124
Epoch [42/1000], Loss: 1.3144
Epoch [44/1000], Loss: 1.2658
Epoch [46/1000], Loss: 1.3357
Epoch [48/1000], Loss: 1.3493
Epoch [50/1000], Loss: 1.2778
Epoch [52/1000], Loss: 1.2938
Epoch [54/1000], Loss: 1.2770
Epoch [56/1000], Loss: 1.3062
Epoch [58/1000], Loss: 1.2839
Epoch [60/1000], Loss: 1.2408
Epoch [62/1000], Loss: 1.2253
Epoch [64/1000], Loss: 1.2268
Epoch [66/1000], Loss: 1.3000
Epoch [68/1000

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Regression with NNKNN

In [ ]:
unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")

Unique values: [-1.3415393  -0.44717973  0.44717973  1.3415393 ]
Counts: [1651 1651 1651 1651]
Xs.size(): torch.Size([6604, 162])


In [ ]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, _, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)

  return best_accuracy, accuracy, top_k_average_accuracy, model

In [ ]:
# prompt: load_model_reg()

def load_model_reg(X_train,y_train,cfg):
  # Define the model architecture
  model = Rmodel.NN_k_NN_regression(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      cfg.class_weight_sharing,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model


In [ ]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Epoch [2/1000], Loss: 1.1363
Epoch [4/1000], Loss: 1.0000
Epoch [6/1000], Loss: 0.9063
Epoch [8/1000], Loss: 0.9325
Epoch [10/1000], Loss: 1.0979
Epoch [12/1000], Loss: 1.0777
Epoch [14/1000], Loss: 1.0533
Epoch [16/1000], Loss: 1.0521
Epoch [18/1000], Loss: 0.9599
Epoch [20/1000], Loss: 1.2118
Epoch [22/1000], Loss: 1.0878
Epoch [24/1000], Loss: 1.0351
Epoch [26/1000], Loss: 0.9030
Epoch [28/1000], Loss: 0.9901
Epoch [30/1000], Loss: 1.0564
Epoch [32/1000], Loss: 1.0790
Epoch [34/1000], Loss: 1.1429
Epoch [36/1000], Loss: 1.1163
Epoch [38/1000], Loss: 1.1742
Epoch [40/1000], Loss: 0.9303
Epoch [42/1000], Loss: 0.8624
Epoch [44/1000], Loss: 0.9859
Epoch [46/1000], Loss: 0.9060
Epoch [48/1000], Loss: 0.9122
Epoch [50/1000], Loss: 0.9942
Epoch [52/1000], Loss: 0.8828
Epoch [54/1000], Loss: 0.8170
Epoch [56/1000], Loss: 0.8536
Epoch [58/1000], Loss: 0.9334
Epoch [60/1000], Loss: 1.0324
Epoch [62/1000], Loss: 0.8176
Epoch [64/1000], Loss: 0.9927
Epoch [66/1000], Loss: 0.9490
Epoch [68/1000

# Results Interpretation

In [55]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [56]:
print_model_features(model)

fa_layer.f1weight
tensor([2.9837e+00, 2.2470e+00, 9.1954e-01, 2.8684e+00, 1.4802e+00, 5.3294e+00,
        3.9646e+00, 3.7706e+00, 9.4106e-01, 9.2749e-01, 1.3207e+00, 5.4343e+00,
        6.1051e-01, 5.4291e-01, 1.1546e+00, 7.4379e-01, 4.0938e+00, 4.7726e+00,
        9.4088e-01, 1.2448e-01, 1.0540e+00, 2.8525e-01, 4.9242e+00, 4.3309e+00,
        3.0648e+00, 4.3516e+00, 1.3300e+00, 2.9637e+00, 6.5936e+00, 3.8686e-01,
        1.8404e+00, 2.2621e+00, 3.4485e+00, 3.2847e+00, 8.8343e-01, 6.3579e-01,
        5.0706e+00, 1.5870e-01, 7.8502e+00, 4.7005e+00, 2.7071e+00, 3.8592e+00,
        3.8657e+00, 7.1171e-01, 3.9717e+00, 3.7286e-01, 5.9990e+00, 1.2326e+00,
        2.5626e+00, 2.1821e+00, 6.1737e-01, 8.2093e-01, 4.7545e+00, 4.3071e+00,
        4.6586e+00, 1.7714e+00, 3.7991e+00, 4.4027e+00, 6.3810e-03, 9.3399e-01,
        5.3896e+00, 2.2530e+00, 5.5890e+00, 5.4393e+00, 8.6380e-01, 5.2013e+00,
        2.9872e-01, 4.1131e+00, 6.1163e+00, 4.5210e+00, 3.8321e+00, 5.1768e+00,
        4.8790e+00, 5.

In [57]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test)

In [58]:
predicted_class

tensor([3, 3, 2, 1, 0, 0, 3, 1, 3, 2, 0, 1, 3, 1, 3, 2, 3, 0, 3, 3, 1, 0, 2, 3,
        3, 0, 1, 2, 0, 0, 1, 3, 2, 3, 0, 3, 1, 1, 1, 3, 2, 3, 3, 2, 3, 3, 0, 3,
        3, 0, 3, 3, 3, 2, 1, 3, 1, 1, 3, 3, 1, 1, 3, 0, 3, 3, 0, 0, 3, 1, 1, 3,
        3, 3, 1, 1, 3, 1, 3, 1, 1, 0, 0, 2, 2, 1, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3,
        0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 1, 2, 3, 3, 1, 3, 3, 3, 3, 0, 3,
        3, 0, 3, 3, 3, 3, 0, 3, 1, 0, 3, 3, 2, 3, 3, 1, 3, 1, 3, 0, 0, 3, 3, 1,
        1, 3, 3, 3, 1, 0, 3, 0, 3, 2, 2, 3, 3, 0, 3, 0, 3, 0, 2, 0, 1, 2, 3, 3,
        3, 0, 3, 3, 3, 0, 2, 3, 1, 0, 1, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 2, 3,
        1, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 1, 0, 3, 3, 3, 1,
        1, 3, 2, 1, 2, 3, 3, 3, 1, 2, 3, 0, 3, 3, 2, 0, 3, 3, 2, 3, 3, 1, 0, 3,
        2, 3, 3, 0, 3, 1, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0,
        0, 2, 3, 3, 0, 0, 3, 1, 3, 0, 3, 3, 2, 3, 3, 3, 0, 3, 1, 2, 0, 3, 2, 3,
        3, 2, 3, 1, 2, 3, 2, 0, 0, 3, 3,

In [59]:
y_test

tensor([0, 1, 1, 3, 0, 1, 3, 1, 0, 0, 0, 2, 1, 2, 3, 0, 3, 3, 2, 2, 0, 1, 1, 3,
        1, 0, 0, 2, 0, 0, 3, 2, 1, 3, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 0, 2, 1,
        2, 3, 2, 0, 3, 2, 1, 2, 3, 3, 1, 2, 1, 3, 0, 2, 1, 2, 1, 0, 3, 1, 0, 0,
        2, 2, 2, 0, 2, 2, 2, 3, 2, 1, 0, 2, 2, 3, 3, 2, 1, 0, 0, 2, 2, 2, 3, 1,
        3, 2, 2, 3, 3, 1, 3, 3, 1, 0, 0, 1, 0, 1, 0, 1, 3, 3, 1, 1, 3, 2, 0, 2,
        3, 2, 3, 2, 1, 1, 0, 2, 0, 0, 2, 3, 2, 3, 1, 0, 0, 2, 1, 3, 1, 3, 3, 0,
        0, 1, 0, 2, 0, 0, 0, 0, 3, 0, 2, 1, 1, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 3,
        2, 1, 3, 0, 3, 3, 1, 0, 0, 0, 0, 3, 3, 2, 1, 1, 3, 1, 3, 0, 3, 2, 1, 0,
        3, 3, 3, 0, 3, 3, 2, 1, 1, 1, 0, 3, 2, 2, 2, 1, 0, 3, 1, 0, 2, 1, 3, 0,
        2, 1, 1, 0, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 2, 3, 3, 3, 2, 1, 0, 1,
        3, 1, 1, 0, 1, 2, 1, 3, 0, 0, 3, 0, 0, 1, 1, 2, 1, 2, 1, 3, 0, 1, 3, 2,
        0, 2, 0, 3, 1, 0, 1, 1, 2, 1, 0, 2, 3, 2, 0, 1, 0, 3, 1, 1, 1, 0, 1, 3,
        2, 0, 2, 3, 1, 3, 0, 2, 0, 1, 1,

In [60]:
# prompt: accuracy comparing predicted_class and y_test

accuracy = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
print("Accuracy:", accuracy)


Accuracy: 0.3010590015128593


In [61]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [62]:
X_test[0]

tensor([-0.0326, -0.0784, -0.0784, -0.0326, -0.0326, -0.0784, -0.0784, -0.0326,
        -0.0784, -0.0784, -0.0784, -0.1242, -0.1242, -0.0784, -0.1242, -0.0784,
        -0.0784, -0.0784, -0.1242, -0.0784, -0.0784, -0.0784, -0.0784, -0.0784,
        -0.0784, -0.1242, -0.1242, -0.1700, -0.1700, -0.1700, -0.1242, -0.1700,
        -0.1242, -0.1700, -0.1700, -0.1700, -0.1242, -0.1242, -0.1700, -0.1700,
        -0.1700, -0.1242, -0.1700, -0.1242, -0.1700, -0.1700, -0.1700,  0.0591,
         0.0591,  0.0591,  0.0591,  0.0591, -0.0784, -0.0784, -0.0784, -0.0784,
        -0.0784, -0.0784, -0.0784, -0.1242, -0.0784, -0.0784, -0.0784, -0.0784,
        -0.0784, -0.0784, -0.0784, -0.1242, -0.0784, -0.0784, -0.0784, -0.1700,
        -0.1700, -0.1242, -0.1700, -0.1700, -0.1700, -0.1700, -0.1700, -0.1700,
        -0.1700, -0.1700, -0.0326, -0.1700, -0.0784, -0.1700,  0.0591,  4.3198,
         0.0133,  0.0133,  0.0133, -0.1242,  0.0133, -0.1242,  0.0133,  0.0133,
         0.0133,  0.0133,  0.0133,  0.01

In [63]:
y_test[0]

tensor(0)

In [64]:
X_train[top_case_indices[0][0]]

tensor([-0.0326,  0.0133, -0.0326, -0.1242, -0.1242, -0.0784, -0.1242,  0.0133,
        -0.1242, -0.1242, -0.0784, -0.1242, -0.1242, -0.1242, -0.0784, -0.0784,
        -0.1242, -0.0784, -0.1242, -0.1242, -0.0784, -0.0784, -0.0784, -0.0784,
        -0.1242, -0.1700, -0.1242, -0.1242, -0.1700, -0.1700, -0.1700, -0.1700,
        -0.1242, -0.1700, -0.1700, -0.1700, -0.1700, -0.1242, -0.1242, -0.1242,
        -0.1242, -0.1242, -0.1242, -0.1242, -0.1700, -0.1700, -0.1700,  0.0133,
        -0.0326,  0.0133, -0.0326, -0.0326, -0.1242, -0.0784, -0.1242, -0.0784,
        -0.0784, -0.0784, -0.0784, -0.0784, -0.0784, -0.0784, -0.1242, -0.0784,
        -0.1242, -0.1242, -0.0784, -0.1242, -0.0784, -0.0784, -0.1242, -0.1242,
        -0.1700, -0.1700, -0.1700, -0.1700, -0.1700, -0.1700, -0.1700, -0.1700,
        -0.1700, -0.1700, -0.0326, -0.1700, -0.0784, -0.1700,  0.0133,  4.3198,
         0.0133,  0.0133,  0.0133,  0.0133,  0.0133, -0.1242,  0.0133,  0.0133,
         0.0133,  0.0133,  0.0133,  0.01

In [65]:
y_train[top_case_indices[0][0]]

tensor(0)

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [66]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

tensor(7.0508)

In [67]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

26.561165


TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [68]:
y_train[top_case_indices[0]]

tensor([0, 0, 2, 3, 0])

In [69]:
knn.predict(X_test)[0]

0

In [70]:
indices = knn.kneighbors(X_test)[1][0]

In [71]:
y_train[indices]

tensor([0, 0, 2, 1, 0])